In [31]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

from datetime import datetime


Python version
3.10.4 (main, Mar 31 2022, 03:38:35) [Clang 12.0.0 ]
Version info.
sys.version_info(major=3, minor=10, micro=4, releaselevel='final', serial=0)


In [32]:
# variable holding the query payload
query = """
{
    registrations(first:1000, orderBy:registrationDate, orderDirection:desc){
        domain{
            name
        }
        registrant {
            id
        }
        registrationDate
        cost
        expiryDate
    }
}
"""

def get_data(query):
    """This function posts a request to make an API call to ENS Subgraph URL
    parameters:
    ------------
    query: payload containing specific data we need
    return:
    -------
    response.json(): queried data in JSON format
    """

    response = requests.post('https://api.thegraph.com/subgraphs/name/ensdomains/ens'
                             '',
                             json={"query":query})

    if response.status_code == 200: # code 200 means no errors 
        return response.json()
    else: # if errors, print the error code for debugging
        raise Exception("Query failed with return code {}".format(response.staus_code))


In [33]:
import pandas as pd

In [34]:
import requests

In [35]:
data = get_data(query)
display(data)

{'data': {'registrations': [{'domain': {'name': 'greendeposit.eth'},
    'registrant': {'id': '0x14f416c03e1215d8453c9a0ed2930339b1079d77'},
    'registrationDate': '1664116019',
    'cost': '30624423464918767',
    'expiryDate': '1916571635'},
   {'domain': {'name': '🎅🏻👶🏻.eth'},
    'registrant': {'id': '0xc6439cfc38bf2f354ec23e02a5307996cbb34468'},
    'registrationDate': '1664116007',
    'cost': '9677317007489029',
    'expiryDate': '1666609006'},
   {'domain': {'name': 'rektgurus.eth'},
    'registrant': {'id': '0xc50b004ecb83a949ee79a5263458eb6377fdcd8f'},
    'registrationDate': '1664116007',
    'cost': '7656105866229691',
    'expiryDate': '1727229911'},
   {'domain': {'name': 'nela💕.eth'},
    'registrant': {'id': '0xed70395bb8efd758248abd07c3cdc116c32957ea'},
    'registrationDate': '1664116007',
    'cost': '7656105866229691',
    'expiryDate': '1727229911'},
   {'domain': {'name': 'galafam.eth'},
    'registrant': {'id': '0x461ef75dc65838683c2e0c52c86b67108b58215a'},
    '

In [36]:
from subgrounds import Subgrounds

In [37]:
sg=Subgrounds()

In [38]:
# load ENS subgraph 
ens = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/ensdomains/ens')

In [39]:
from subgrounds.subgraph import SyntheticField

# registrationdate synthetic field
ens.Registration.registrationdate = SyntheticField(
    lambda registrationDate: str(datetime.fromtimestamp(registrationDate)),
    SyntheticField.STRING,
    ens.Registration.registrationDate
)

# expirydate synthetic field
ens.Registration.expirydate = SyntheticField(
    lambda expiryDate: str(datetime.fromtimestamp(expiryDate)),
    SyntheticField.STRING,
    ens.Registration.expiryDate
)


In [40]:
# Select the latest 1000 registration names by registration datetime
registrations = ens.Query.registrations(
    first=1000, # latest 1000 registrations
    orderBy=ens.Registration.registrationDate, # order registrations by time
    orderDirection="desc" # latest registration data will be first
)

field_paths = [
    registrations.domain.name, # ens domain like "vitalik.eth"
    registrations.registrant.id, # hexadecimal eth address
    registrations.registrationdate, # time in epoch format
    registrations.cost, # price for registration
    registrations.expirydate # expiry date of domain
]


In [41]:
# get data
df = sg.query_df(field_paths)

# print the first five results
df.head()

,registrations_domain_name,registrations_registrant_id,registrations_registrationdate,registrations_cost,registrations_expirydate
0,greendeposit.eth,0x14f416c03e1215d8453c9a0ed2930339b1079d77,2022-09-25 16:26:59,30624423464918767,2030-09-25 15:00:35
1,🎅🏻👶🏻.eth,0xc6439cfc38bf2f354ec23e02a5307996cbb34468,2022-09-25 16:26:47,9677317007489029,2022-10-24 12:56:46
2,rektgurus.eth,0xc50b004ecb83a949ee79a5263458eb6377fdcd8f,2022-09-25 16:26:47,7656105866229691,2024-09-25 04:05:11
3,nela💕.eth,0xed70395bb8efd758248abd07c3cdc116c32957ea,2022-09-25 16:26:47,7656105866229691,2024-09-25 04:05:11
4,galafam.eth,0x461ef75dc65838683c2e0c52c86b67108b58215a,2022-09-25 16:26:23,19140264665574229,2027-09-25 21:32:23
